# LG Aimers 35→7 Tweedie(LightGBM) — 함수 최소화 디버깅 노트북
단계별로 편집·재실행 가능. 핵심 로직 동일.
날씨 평균기온, 최고기온 적용

In [ ]:
!pip install lightgbm

## 0. 공통 설정

In [1]:

import os, re, glob, unicodedata, warnings
import numpy as np, pandas as pd
from sklearn.multioutput import MultiOutputRegressor
import lightgbm as lgb
warnings.filterwarnings("ignore")

# 고정 설정
SEED_BASE = 42
np.random.seed(SEED_BASE)
PREDICT, INPUT_WINDOW_DAYS = 7, 35
DATE, KEY, TARGET = "영업일자","영업장명_메뉴명","매출수량"

SEEDS_FULL, N_ESTIMATORS_FULL, ANCHOR_K_FULL = 1, 1200, 7
SEEDS_SEARCH, N_ESTIMATORS_SEARCH, ANCHOR_K_SEARCH = 1, 400, 3

NON_SELLING_MIN_DAYS = 14
APPLY_METRIC_AWARE_WSMAPE   = True
APPLY_METRIC_AWARE_WK_HOLOV = True
FORCE_ACTIVE_MIN1 = True
FORCE_GLOBAL_MIN1 = True

CATEGORICAL_COLS = ["key_encoded","month","dayofweek"]
EXTRA_GROUP_KEYS = {"연회장_Cass Beer", "연회장_Regular Coffee"}
BRUNCH_WD = "미라시아_브런치(대인) 주중"
BRUNCH_WE = "미라시아_브런치(대인) 주말"

# 작은 헬퍼(불가피)
to_datetime_norm = lambda s: pd.to_datetime(s, errors="coerce").dt.normalize()

def smape_comp(y_true, y_pred):
    y_true = np.asarray(y_true, float); y_pred = np.asarray(y_pred, float)
    m = (y_true != 0)
    if not np.any(m): return np.nan
    yt, yp = y_true[m], y_pred[m]
    return np.mean(2*np.abs(yp-yt)/(np.abs(yt)+np.abs(yp))) * 100

def round_nonneg(pred, thr):
    pred = np.maximum(pred, 0.0)
    frac = pred - np.floor(pred)
    out  = np.where(frac >= thr, np.floor(pred)+1, np.floor(pred))
    return np.maximum(out, 0.0)


## 1. 데이터 로드

In [2]:
group_m =  pd.read_csv('TRAIN_group.csv')
group_m

,영업일자,느티나무 셀프BBQ,담하,라그로타,미라시아,카페테리아,포레스트릿,화담숲주막,화담숲카페
0,2023-01-01,0,0,0,2,0,0,0,0
1,2023-01-02,0,6,0,2,0,0,0,0
2,2023-01-03,4,4,0,1,2,0,0,0
3,2023-01-04,0,2,0,3,1,0,0,0
4,2023-01-05,4,2,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...
527,2024-06-11,19,6,0,3,4,0,0,0
528,2024-06-12,10,13,10,1,1,0,0,0
529,2024-06-13,15,4,0,4,12,0,0,0
530,2024-06-14,21,8,0,5,6,0,0,0


In [3]:
weather_m  = pd.read_csv('TRAIN_weather.csv')
weather_m

,일시,평균기온(℃),최고기온(℃),최고기온시각,최저기온(℃),최저기온시각,일교차,강수량(mm)
0,2023-01-01,-1.7,5.5,13:55,-7.7,08:14,13.2,NaN
1,2023-01-02,-4.1,1.3,14:29,-9.0,07:45,10.3,NaN
2,2023-01-03,-6.2,1.7,14:58,-14.3,08:03,16.0,NaN
3,2023-01-04,-3.8,3.9,15:34,-10.8,03:28,14.7,NaN
4,2023-01-05,-5.3,2.3,15:31,-10.6,05:58,12.9,NaN
...,...,...,...,...,...,...,...,...
527,2024-06-11,25.4,31.8,14:57,20.1,01:51,11.7,NaN
528,2024-06-12,24.9,33.3,15:36,16.4,05:16,16.9,NaN
529,2024-06-13,25.5,34.3,15:08,16.9,05:42,17.4,NaN
530,2024-06-14,25.8,33.8,14:25,17.7,05:18,16.1,NaN


In [4]:
ski_m = pd.read_csv('TRAIN_ski.csv')
ski_m

,영업일자,8시,9시,10시,11시,12시,13시,14시,15시,16시,...,23시,24시,1시,2시,3시,4시,5시,6시,7시,1일내장객
0,2023-01-01,585.0,552.0,947.0,1284.0,1658.0,1815.0,1970.0,1746.0,1467.0,...,813.0,577.0,268.0,57.0,19.0,6.0,3.0,0.0,3.0,5759.0
1,2023-01-02,387.0,421.0,793.0,1089.0,1338.0,1598.0,1716.0,1494.0,1279.0,...,1122.0,834.0,391.0,91.0,27.0,9.0,4.0,0.0,1.0,4574.0
2,2023-01-03,396.0,375.0,710.0,976.0,1253.0,1491.0,1580.0,1387.0,1224.0,...,1177.0,842.0,359.0,100.0,33.0,11.0,1.0,1.0,1.0,4585.0
3,2023-01-04,404.0,392.0,718.0,983.0,1198.0,1411.0,1558.0,1415.0,1203.0,...,1189.0,886.0,373.0,107.0,34.0,13.0,5.0,1.0,0.0,4634.0
4,2023-01-05,436.0,435.0,750.0,1086.0,1355.0,1683.0,1849.0,1658.0,1471.0,...,1336.0,916.0,443.0,121.0,40.0,20.0,15.0,2.0,0.0,5371.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
527,2024-06-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
528,2024-06-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
529,2024-06-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
530,2024-06-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# train 로드(인코딩 폴백)
try:
    train = pd.read_csv('train.csv')
except UnicodeDecodeError:
    train = pd.read_csv('train.csv', encoding="cp949")

# 컬럼 표준화(필요시)
def _canon(s):
    return re.sub(r"\s+","", unicodedata.normalize("NFKC", str(s).replace("\ufeff","").replace("\xa0"," ").strip()).lower()).replace("_","").replace("-","")
orig = list(train.columns); cmap = {c:_canon(c) for c in orig}
date_alias   = {_canon(x) for x in ["영업일자","일자","날짜","date"]}
key_single   = {_canon(x) for x in ["영업장명_메뉴명","영업장명메뉴명","key","메뉴키"]}
store_alias  = {_canon(x) for x in ["영업장명","매장명","지점명","점포명","매장","영업장"]}
menu_alias   = {_canon(x) for x in ["메뉴명","상품명","제품명","메뉴"]}
targ_alias   = {_canon(x) for x in ["매출수량","판매수량","수량","qty","판매량"]}


date_col   = next((c for c in orig if cmap[c] in date_alias), None)
target_col = next((c for c in orig if cmap[c] in targ_alias), None)
key_col    = next((c for c in orig if cmap[c] in key_single), None)
if key_col is None:
    store_col = next((c for c in orig if cmap[c] in store_alias), None)
    menu_col  = next((c for c in orig if cmap[c] in menu_alias ), None)
    train[KEY] = train[store_col].astype(str).str.strip() + "_" + train[menu_col].astype(str).str.strip()
else:
    if key_col != KEY: train = train.rename(columns={key_col:KEY})
if date_col != DATE:  train = train.rename(columns={date_col:DATE})
if target_col != TARGET: train = train.rename(columns={target_col:TARGET})

# 타입 정리
train[DATE] = to_datetime_norm(train[DATE])
train[TARGET] = pd.to_numeric(train[TARGET], errors="coerce").clip(lower=0)

print(train.shape, "keys:", train[KEY].nunique(), "기간:", train[DATE].min().date(), "→", train[DATE].max().date())
train.head(3)


(88844, 3) keys: 167 기간: 2023-01-01 → 2024-06-15


,영업일자,영업장명_메뉴명,매출수량
0,2023-01-01,느티나무 셀프BBQ_1인 수저세트,0
1,2023-01-02,느티나무 셀프BBQ_1인 수저세트,0
2,2023-01-03,느티나무 셀프BBQ_1인 수저세트,0


In [6]:
# weather_m는 이미 read_csv됨
norm = lambda s: pd.to_datetime(s, errors="coerce").dt.normalize()
w = weather_m.rename(columns={"일시": DATE}).copy()
w[DATE] = norm(w[DATE])
# 컬럼 자동 탐색(최고/평균 + 기온 포함)
col_tmax = next(c for c in w.columns if ("최고" in c and "기온" in c))
col_tavg = next(c for c in w.columns if ("평균" in c and "기온" in c))
w_day = (w[[DATE, col_tmax, col_tavg]]
         .groupby(DATE, as_index=False).mean(numeric_only=True)
         .rename(columns={col_tmax:"wx_tmax", col_tavg:"wx_tavg"}))

In [7]:
g = group_m.copy()
g[DATE] = norm(g[DATE])

# 겨울 시즌 플래그
winter_m = g[[DATE]].drop_duplicates().copy()
winter_m[DATE] = norm(winter_m[DATE])

winter_m["month"] = winter_m[DATE].dt.month
winter_m["is_ski_season"] = winter_m["month"].isin([12,1,2]).astype(int)
    
# 시즌 상대적 위치 (단일 시즌 기준)
season_mask = winter_m["is_ski_season"] == 1
if season_mask.any():
    season_start = winter_m.loc[season_mask, DATE].min()
    season_end   = winter_m.loc[season_mask, DATE].max()
    season_len   = (season_end - season_start).days + 1
    winter_m["day_of_season"] = (
        (winter_m[DATE] - season_start).dt.days / season_len
    ).where(season_mask, 0.0)
else:
    winter_m["day_of_season"] = 0.0

In [8]:
# group_m 기준으로 is_forest_event 생성
g = group_m.rename(columns={"영업일자": DATE}).copy()
g[DATE] = pd.to_datetime(g[DATE]).dt.normalize()

# 카페테리아 + 포레스트릿 합산
col_a = next(c for c in g.columns if "카페테리아" in c)
col_b = next(c for c in g.columns if "포레스트릿" in c)
g["cf_sum"] = g[col_a] + g[col_b]

# 이진화
forest_m = (g[[DATE, "cf_sum"]]
            .groupby(DATE, as_index=False).max(numeric_only=True))
forest_m["is_forest_event"] = (forest_m["cf_sum"] > 0).astype(int)
forest_m = forest_m[[DATE, "is_forest_event"]]

## 2. 전처리
- 평일 공휴일 세트 구성(train+test 범위)
- 평일 공휴일 급등 클램프(train만)

In [9]:
# test 캘린더 로드
test_files = sorted(glob.glob('test/TEST_*.csv'))
test_list = []
for p in test_files:
    try:
        t = pd.read_csv(p)
    except UnicodeDecodeError:
        t = pd.read_csv(p, encoding="cp949")
    # 표준 가정: DATE, KEY, TARGET 이름 동일 혹은 위에서 감지 로직 재사용 필요시 동일 처리 가능
    if DATE not in t.columns:
        # 간단 치환(대회 제공 포맷 가정). 필요시 수동 수정.
        pass
    t[DATE] = to_datetime_norm(t[DATE])
    test_list.append(t)

# HOL_WEEKDAY_SET 구성
dmin = min([train[DATE].min()] + [t[DATE].min() for t in test_list])
dmax = max([train[DATE].max()] + [t[DATE].max() for t in test_list]) + pd.Timedelta(days=PREDICT+14)

manual_days = pd.to_datetime([
    "2023-01-23","2023-01-24","2023-03-01","2023-05-05","2023-05-29","2023-06-06","2023-08-15",
    "2023-09-28","2023-09-29","2023-10-02","2023-10-03","2023-10-09","2023-12-25",
    "2024-01-01","2024-02-09","2024-02-12","2024-03-01","2024-04-10","2024-05-06","2024-05-15",
    "2024-06-06","2024-08-15","2024-09-16","2024-09-17","2024-09-18","2024-10-01","2024-10-03","2024-10-09","2024-12-25",
    "2025-01-01","2025-01-27","2025-01-28","2025-01-29","2025-01-30","2025-03-03",
    "2025-05-05","2025-05-06","2025-06-03","2025-06-06","2025-08-15","2025-10-03","2025-10-09","2025-12-25",
    "2026-01-01"
]).normalize()

try:
    import holidays as _hol
    years = sorted({d.year for d in pd.date_range(dmin, dmax, freq="D")})
    kr = _hol.KR(years=years)
    hol = pd.DatetimeIndex([pd.Timestamp(k) for k in kr]).normalize()
    hol = pd.DatetimeIndex(sorted(set(hol) | set(manual_days)))
except Exception:
    hol = pd.DatetimeIndex(sorted(set(manual_days)))
hol = hol[(hol>=dmin)&(hol<=dmax)]
HOL_WEEKDAY_SET = set(hol[hol.dayofweek<5])
print("HOL_WEEKDAY_SET size:", len(HOL_WEEKDAY_SET))

# 평일 공휴일 급등 완화(train만)
x = train.copy().sort_values([KEY, DATE]).reset_index(drop=True)
x["dow"] = x[DATE].dt.dayofweek
x["is_nonhol_weekday"] = ((x["dow"]<5) & (~x[DATE].isin(HOL_WEEKDAY_SET))).astype(int)
x["is_hol_wk"]         = (x[DATE].isin(HOL_WEEKDAY_SET)).astype(int)
x["__val__"] = x[TARGET].astype(float)

outs = []
for k, g in x.groupby(KEY, sort=False):
    if (BRUNCH_WD in str(k)) or (BRUNCH_WE in str(k)):
        outs.append(g); continue
    g = g.copy()
    g["med_wd_dow"] = np.nan
    for d in range(5):
        idx = (g["dow"]==d) & (g["is_nonhol_weekday"]==1)
        s = g.loc[idx,"__val__"]
        med = s.shift(1).rolling(window=4, min_periods=2).median()
        g.loc[idx,"med_wd_dow"] = med.values
    is_metric_target = (("단체" in str(k)) or (str(k) in EXTRA_GROUP_KEYS)) and not ((BRUNCH_WD in str(k)) or (BRUNCH_WE in str(k)))
    if is_metric_target:
        thr_up = (1.0+0.25)*g["med_wd_dow"]
        adj = ((g["is_hol_wk"]==1)&np.isfinite(g["med_wd_dow"])&(g["__val__"]>thr_up))
        g.loc[adj,"__val__"] = np.maximum(0.0, np.round((1.0+0.10)*g.loc[adj,"med_wd_dow"]))
    g = g.drop(columns=["med_wd_dow"])
    outs.append(g)

train_pre = pd.concat(outs, axis=0).drop(columns=["dow","is_nonhol_weekday","is_hol_wk"])
train_pre[TARGET] = train_pre["__val__"].astype(float).clip(lower=0)
train_pre = train_pre.drop(columns="__val__").sort_values([KEY, DATE]).reset_index(drop=True)

print("train_pre:", train_pre.shape, "기간:", train_pre[DATE].min().date(), "→", train_pre[DATE].max().date())
train_pre.head(3)

HOL_WEEKDAY_SET size: 39
train_pre: (88844, 3) 기간: 2023-01-01 → 2024-06-15


,영업일자,영업장명_메뉴명,매출수량
0,2023-01-01,느티나무 셀프BBQ_1인 수저세트,0.0
1,2023-01-02,느티나무 셀프BBQ_1인 수저세트,0.0
2,2023-01-03,느티나무 셀프BBQ_1인 수저세트,0.0


In [10]:
train_pre = train_pre.merge(w_day, on=DATE, how="left")
train_pre = train_pre.merge(winter_m, on=DATE, how="left")
train_pre = train_pre.merge(forest_m, on=DATE, how="left")

In [11]:
train_pre.head()

,영업일자,영업장명_메뉴명,매출수량,wx_tmax,wx_tavg,month,is_ski_season,day_of_season,is_forest_event
0,2023-01-01,느티나무 셀프BBQ_1인 수저세트,0.0,5.5,-1.7,1,1,0.000000,0
1,2023-01-02,느티나무 셀프BBQ_1인 수저세트,0.0,1.3,-4.1,1,1,0.002353,0
2,2023-01-03,느티나무 셀프BBQ_1인 수저세트,0.0,1.7,-6.2,1,1,0.004706,1
3,2023-01-04,느티나무 셀프BBQ_1인 수저세트,0.0,3.9,-3.8,1,1,0.007059,1
4,2023-01-05,느티나무 셀프BBQ_1인 수저세트,0.0,2.3,-5.3,1,1,0.009412,1


## 3. 피처 추가
- 랙·롤링·EWM·푸리에·미래 캘린더
- KEY 인코딩은 학습에서만 fit

In [12]:
room_m = pd.read_csv('TRAIN_room.csv')
room_type = pd.read_csv('room_type.csv')

room_dict = {}
for _type, _num in zip(room_type['객실타입'], room_type['기준인원']):
    room_dict.update({_type:_num})

today_num = []
for i in range(len(room_m)):
    a = room_m.iloc[i]
    _sum = 0
    for _type in room_dict.keys():
        _sum += a[_type]*room_dict[_type]

    today_num.append(_sum)

room_m['총방문객수'] = today_num

# df2의 영업일자를 datetime으로 변환
room_m['영업일자'] = pd.to_datetime(room_m['영업일자'])

train_pre = train_pre.merge(room_m[[DATE, '총방문객수']], on=DATE, how='left')

In [13]:
# 학습용 피처 생성
df = train_pre.copy()
df = df.sort_values([KEY, DATE]).reset_index(drop=True)

df["month"] = df[DATE].dt.month
df["dayofweek"] = df[DATE].dt.dayofweek
df["day"] = df[DATE].dt.day
df["dayofyear"] = df[DATE].dt.dayofyear
df["is_month_end"] = df[DATE].dt.is_month_end.astype(int)
df["is_quarter_end"] = df[DATE].dt.is_quarter_end.astype(int)
df["is_weekend"] = (df["dayofweek"]>=5).astype(int)

# 미래 캘린더
for h in range(1, PREDICT+1):
    dth = df[DATE] + pd.to_timedelta(h, unit="D")
    df[f"dow_h{h}"]        = dth.dt.dayofweek
    df[f"is_weekend_h{h}"] = (df[f"dow_h{h}"]>=5).astype(int)
    df[f"is_wkhol_h{h}"]   = dth.isin(pd.DatetimeIndex(HOL_WEEKDAY_SET)).astype(int)

# 랙/롤링
for lag in [7,14,21]:
    df[f"lag_{lag}"] = df.groupby(KEY)[TARGET].shift(lag)
for w in [7,14]:
    df[f"rolling_mean_{w}"]   = df.groupby(KEY)[TARGET].transform(lambda x: x.shift(1).rolling(w).mean())
    df[f"rolling_std_{w}"]    = df.groupby(KEY)[TARGET].transform(lambda x: x.shift(1).rolling(w).std())
    df[f"rolling_median_{w}"] = df.groupby(KEY)[TARGET].transform(lambda x: x.shift(1).rolling(w).median())
for lag in [1,2,3,4,5,6]:
    df[f"lag_{lag}"] = df.groupby(KEY)[TARGET].shift(lag)

df["rolling_mean_21"] = df.groupby(KEY)[TARGET].transform(lambda x: x.shift(1).rolling(21).mean())
df["ratio_mean7_21"]  = df["rolling_mean_7"]/df["rolling_mean_21"]
df["ratio_mean14_21"] = df["rolling_mean_14"]/df["rolling_mean_21"]

def _slope_raw(v):
    m = np.isfinite(v); v = v[m]; n=v.size
    if n<2: return np.nan
    x = np.arange(n, dtype=float); xm, ym = x.mean(), v.mean()
    denom = ((x-xm)**2).sum();
    if denom==0: return 0.0
    return float(((x-xm)*(v-ym)).sum()/denom)

df["slope_7"]  = df.groupby(KEY)[TARGET].transform(lambda x: x.shift(1).rolling(7).apply(_slope_raw, raw=True))
df["slope_14"] = df.groupby(KEY)[TARGET].transform(lambda x: x.shift(1).rolling(14).apply(_slope_raw, raw=True))

# 무판매 장기구간 플래그
def _flag_long_zero_block(g, min_days):
    is_zero = (g[TARGET]==0).astype(int)
    block_id = (is_zero != is_zero.shift()).cumsum()
    block_len = is_zero.groupby(block_id).transform("sum")
    return ((is_zero==1)&(block_len>=min_days)).astype(int)
df["is_long_zero_block"] = df.groupby(KEY, sort=False).apply(lambda g: _flag_long_zero_block(g, NON_SELLING_MIN_DAYS)).reset_index(level=0, drop=True).astype(int)

# EWM
df["ewm_mean_7"] = df.groupby(KEY)[TARGET].transform(lambda x: x.shift(1).ewm(span=7, adjust=False, min_periods=2).mean())

# 요일 Fourier
x_dow = df["dayofweek"].astype(float)
df["dow_sin"] = np.sin(2*np.pi*x_dow/7.0); df["dow_cos"] = np.cos(2*np.pi*x_dow/7.0)

# KEY 인코딩
cats = pd.Index(sorted(df[KEY].astype(str).unique()))
key2id = {k:i for i,k in enumerate(cats)}
df["key_encoded"] = df[KEY].astype(str).map(key2id).astype("category")

# 타깃 시프팅
for i in range(1, PREDICT+1):
    df[f"target_{i}"] = df.groupby(KEY)[TARGET].shift(-i)
df_trainF = df.dropna().reset_index(drop=True)

# 피처 목록
feat_cols = [
    "month","dayofweek","key_encoded","day","dayofyear","is_month_end","is_quarter_end","is_weekend",
    "rolling_mean_7","rolling_std_7","rolling_median_7",
    "rolling_mean_14","rolling_std_14","rolling_median_14",
    "rolling_mean_21","ratio_mean7_21","ratio_mean14_21",
    "slope_7","slope_14","ewm_mean_7","dow_sin","dow_cos","is_long_zero_block","wx_tmax","wx_tavg","총방문객수",'is_ski_season', 'day_of_season', 'is_forest_event'
] + [c for c in df.columns if re.match(r"^lag_\d+$", c)] + [c for c in df.columns if re.match(r"^(dow_h\d|is_weekend_h\d|is_wkhol_h\d)$", c)]

print("TrainF:", df_trainF.shape, "features:", len(feat_cols))
df_trainF[feat_cols + [f"target_{i}" for i in range(1,PREDICT+1)]].head(3)

TrainF: (68480, 69) features: 59


,month,dayofweek,key_encoded,day,dayofyear,is_month_end,is_quarter_end,is_weekend,rolling_mean_7,rolling_std_7,...,dow_h7,is_weekend_h7,is_wkhol_h7,target_1,target_2,target_3,target_4,target_5,target_6,target_7
0,1,6,0,22,22,0,0,1,1.142857,3.023716,...,6,1,0,4.0,2.0,0.0,0.0,8.0,0.0,8.0
1,1,0,0,23,23,0,0,0,1.142857,3.023716,...,0,0,0,2.0,0.0,0.0,8.0,0.0,8.0,0.0
2,1,1,0,24,24,0,0,0,1.714286,3.147183,...,1,0,0,0.0,0.0,8.0,0.0,8.0,0.0,4.0


In [14]:
# === 리더보드 점수용 지표 ===
import numpy as np
import pandas as pd

def _mask_nonzero(y_true, y_pred):
    y_true = np.asarray(y_true, float)
    y_pred = np.asarray(y_pred, float)
    m = (y_true != 0)
    return y_true[m], y_pred[m]

def smape_frac(y_true, y_pred):
    yt, yp = _mask_nonzero(y_true, y_pred)
    if yt.size == 0: return np.nan
    denom = (np.abs(yt) + np.abs(yp))
    num = 2.0 * np.abs(yp - yt)
    with np.errstate(divide='ignore', invalid='ignore'):
        val = np.where(denom > 0, num / denom, 0.0)
    return float(np.mean(val))

def mae_over_mean(y_true, y_pred):
    yt, yp = _mask_nonzero(y_true, y_pred)
    if yt.size == 0: return np.nan
    mae = np.mean(np.abs(yp - yt))
    ybar = np.mean(yt)
    return float(mae / ybar) if ybar != 0 else np.nan

def rmse_over_mean(y_true, y_pred):
    yt, yp = _mask_nonzero(y_true, y_pred)
    if yt.size == 0: return np.nan
    rmse = np.sqrt(np.mean((yp - yt)**2))
    ybar = np.mean(yt)
    return float(rmse / ybar) if ybar != 0 else np.nan

def r2_pearson(y_true, y_pred):
    yt, yp = _mask_nonzero(y_true, y_pred)
    if yt.size < 2: return np.nan
    corr = np.corrcoef(yt, yp)[0,1]
    if np.isnan(corr): return np.nan
    return float(corr**2)

def leaderboard_score(y_true, y_pred):
    a = smape_frac(y_true, y_pred)
    b = mae_over_mean(y_true, y_pred)
    c = rmse_over_mean(y_true, y_pred)
    d = r2_pearson(y_true, y_pred)
    if np.any(np.isnan([a,b,c,d])): return np.nan
    return 0.25 * ((1 - a/2.0) + (1 - min(1.0, b)) + (1 - min(1.0, c)) + d)

# === WindowCV folds 생성 ===
def make_window_cv(dates, input_window=INPUT_WINDOW_DAYS, horizon=PREDICT,
                   n_folds=5, stride=7, gap=0):
    """
    return: list of (train_end, valid_start, valid_end)
    """
    dates = np.array(sorted(dates))
    last_idx = len(dates) - 1
    out = []
    end_idxs = list(range(last_idx, input_window + horizon - 1, -stride))[:n_folds]
    for ve in end_idxs:
        vs = ve - (horizon - 1)
        te = vs - (gap + 1)
        ts = te - (input_window - 1)
        if ts < 0:
            continue
        out.append((dates[te], dates[vs], dates[ve]))
    out = sorted(out, key=lambda x: x[1])
    return out

In [15]:
# === Tweedie p 앙상블 설정 ===
P_LIST = [1.1, 1.2, 1.3, 1.4]      # 필요시 수정
ENSEMBLE_MODE = "softmax"          # ["equal", "softmax", "linear_pos"]
TAU = 0.5                          # softmax 온도(작을수록 상위 가중 ↑)

def fit_multiout_with_p(X, y, p, seed=SEED_BASE, n_estim=N_ESTIMATORS_FULL):
    base = lgb.LGBMRegressor(
        n_estimators=n_estim, learning_rate=0.04, num_leaves=63,
        feature_fraction=0.8, bagging_fraction=0.8, bagging_freq=1,
        lambda_l1=0.1, lambda_l2=0.1, random_state=seed,
        objective="tweedie", metric="mae", tweedie_variance_power=float(p), verbose=-1
    )
    m = MultiOutputRegressor(base)
    X_ = X.copy()
    for c in ("key_encoded","month","dayofweek"):
        if c in X_.columns: X_[c] = X_[c].astype("category")
    m.fit(X_, y, categorical_feature=[c for c in ("key_encoded","month","dayofweek") if c in X_.columns])
    return m

def make_weights_from_scores(scores_dict, mode="softmax", tau=0.5):
    # scores_dict: {p: mean_LB}
    ps = list(scores_dict.keys())
    s  = np.array([scores_dict[p] for p in ps], float)
    if not np.all(np.isfinite(s)):  # 결측 있으면 균등
        w = np.ones_like(s)/len(s); return dict(zip(ps, w))
    if mode == "equal":
        w = np.ones_like(s)/len(s)
    elif mode == "softmax":
        x = s / max(1e-9, tau)
        x -= x.max()
        w = np.exp(x); w /= w.sum()
    elif mode == "linear_pos":
        s = s - s.min()
        if s.sum() <= 0: w = np.ones_like(s)/len(s)
        else: w = s / s.sum()
    else:
        w = np.ones_like(s)/len(s)
    return dict(zip(ps, w))


In [16]:
# === WindowCV 날짜 ===
dates_sorted = np.array(sorted(df_trainF[DATE].unique()))
folds = make_window_cv(dates_sorted, input_window=INPUT_WINDOW_DAYS,
                       horizon=PREDICT, n_folds=5, stride=7, gap=0)
print("folds:", [(te.date(), vs.date(), ve.date()) for te,vs,ve in folds])

# === p별 CV
cv_tables = {}
mean_lb_by_p = {}

for p in P_LIST:
    fold_metrics = []
    for fi, (train_end, valid_start, valid_end) in enumerate(folds, 1):
        tr_mask = (df_trainF[DATE] <= train_end)
        va_mask = (df_trainF[DATE] >= valid_start) & (df_trainF[DATE] <= valid_end)

        trF = df_trainF.loc[tr_mask].copy()
        vaF = df_trainF.loc[va_mask].copy()
        if len(trF)==0 or len(vaF)==0:
            continue

        Xtr = trF[feat_cols].copy()
        ytr = trF[[f"target_{i}" for i in range(1, PREDICT+1)]].copy()
        model_cv = fit_multiout_with_p(Xtr, ytr, p=float(p), seed=SEED_BASE, n_estim=N_ESTIMATORS_FULL)

        Xva = vaF[feat_cols].copy()
        for c in ("key_encoded","month","dayofweek"):
            if c in Xva.columns: Xva[c] = Xva[c].astype("category")
        y_pred = model_cv.predict(Xva)
        y_true = vaF[[f"target_{i}" for i in range(1, PREDICT+1)]].values

        a = smape_frac(y_true, y_pred)
        b = mae_over_mean(y_true, y_pred)
        c = rmse_over_mean(y_true, y_pred)
        d = r2_pearson(y_true, y_pred)
        L = leaderboard_score(y_true, y_pred)
        fold_metrics.append({"fold": fi, "sMAPE": a, "MAE/mean": b, "RMSE/mean": c, "R2": d, "LB": L})

    cv_df = pd.DataFrame(fold_metrics)
    cv_tables[p] = cv_df
    mean_lb_by_p[p] = float(np.nanmean(cv_df["LB"])) if len(cv_df) else np.nan

# 요약 출력
for p in P_LIST:
    dfp = cv_tables[p]
    print(f"\n[Tweedie p={p}]")
    print(dfp)
    print("mean LB:", np.nanmean(dfp["LB"]) if len(dfp) else np.nan)

# p-가중치 산출
P_WEIGHTS = make_weights_from_scores(mean_lb_by_p, mode=ENSEMBLE_MODE, tau=TAU)
print("\nEnsemble weights by p:", P_WEIGHTS)

# 제출 후처리용 라운딩 임계치 고정
best_thr = 0.130

folds: [(datetime.date(2024, 5, 4), datetime.date(2024, 5, 5), datetime.date(2024, 5, 11)), (datetime.date(2024, 5, 11), datetime.date(2024, 5, 12), datetime.date(2024, 5, 18)), (datetime.date(2024, 5, 18), datetime.date(2024, 5, 19), datetime.date(2024, 5, 25)), (datetime.date(2024, 5, 25), datetime.date(2024, 5, 26), datetime.date(2024, 6, 1)), (datetime.date(2024, 6, 1), datetime.date(2024, 6, 2), datetime.date(2024, 6, 8))]
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] lambda_l2 is set=0.1, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.1
[LightGBM] [Warning] lambda_l1 is set=0.1, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.1
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [

## 5. 모델 학습(전체 학습 데이터)

In [17]:
## 5. p별 전체 학습(models_full 생성)
models_full = {}
X_full = df_trainF[feat_cols].copy()
for c in ("key_encoded","month","dayofweek"):
    if c in X_full.columns: X_full[c] = X_full[c].astype("category")
y_full = df_trainF[[f"target_{i}" for i in range(1, PREDICT+1)]].copy()

for p in P_LIST:
    models_full[p] = fit_multiout_with_p(X_full, y_full, p=float(p), seed=SEED_BASE, n_estim=N_ESTIMATORS_FULL)

# 가중치 정규화 및 검증
s = sum(v for v in P_WEIGHTS.values() if np.isfinite(v))
if s <= 0 or not np.isfinite(s):
    P_WEIGHTS = {p: 1.0/len(P_LIST) for p in P_LIST}
else:
    P_WEIGHTS = {p: float(w)/s for p,w in P_WEIGHTS.items()}
assert len(models_full)>0 and abs(sum(P_WEIGHTS.values())-1)<1e-6
print("fit done for all p:", P_LIST)

[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] lambda_l2 is set=0.1, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.1
[LightGBM] [Warning] lambda_l1 is set=0.1, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.1
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
fit done for all p: [1.1, 1.2, 1.3, 1.4]


## 6. 예측값 도출(Anchor Ensemble, K=7)

In [18]:
# 무판매 장기구간 플래그(학습과 동일)
def _flag_lzb(g):
    iz = (g[TARGET] == 0).astype(int)
    bid = (iz != iz.shift()).cumsum()
    blen = iz.groupby(bid).transform("sum")
    return ((iz == 1) & (blen >= NON_SELLING_MIN_DAYS)).astype(int)
# === 공용 헬퍼 ===
def weekday_holiday_impute(df, hol_set):
    xx = df.copy()
    xx["dow"] = xx[DATE].dt.dayofweek
    xx["is_weekday"] = (xx["dow"] < 5).astype(int)
    xx["is_weekday_hol"] = xx[DATE].isin(hol_set).astype(int)

    outs = []
    for k, g in xx.groupby(KEY, sort=False):
        g = g.sort_values(DATE).copy()
        end_date = g[DATE].max()
        cand = g[(g["is_weekday"]==1) & (g["is_weekday_hol"]==0)].copy()
        all_weekday_zero = (cand[TARGET].sum()==0)

        def _wmean(vals, dates, end_date, decay=0.90, lastweek_window=7, boost=0.50):
            if len(vals)==0: return np.nan
            vals = np.asarray(vals,float); dates = pd.to_datetime(dates)
            delta = (end_date - dates).days.astype(float)
            w = decay ** delta
            w = np.where(dates >= end_date - pd.Timedelta(days=lastweek_window-1), w*(1+boost), w)
            return float(np.sum(w*vals)/np.sum(w))

        for idx, row in g.loc[(g["is_weekday"]==1) & (g["is_weekday_hol"]==1)].iterrows():
            if all_weekday_zero:
                g.at[idx, TARGET] = 0; continue
            pool = cand[cand["dow"]==int(row["dow"])]
            base = _wmean(pool[TARGET].values, pool[DATE].values, end_date) if len(pool) else \
                   _wmean(cand[TARGET].values, cand[DATE].values, end_date)
            t7 = row[DATE] - pd.Timedelta(days=7)
            prev = g[(g[DATE]==t7) & (g["is_weekday"]==1) & (g["is_weekday_hol"]==0)]
            val_t7 = float(prev[TARGET].iloc[0]) if len(prev)==1 else np.nan
            if np.isfinite(base):
                y_hat = base if not np.isfinite(val_t7) else 0.65*base + 0.35*val_t7
                g.at[idx, TARGET] = int(max(0, round(y_hat)))
            else:
                g.at[idx, TARGET] = 0
        outs.append(g)
    return pd.concat(outs, axis=0).drop(columns=["dow","is_weekday","is_weekday_hol"])

def spike_clamp(df, hol_set):
    x2 = df.copy()
    x2["dow"] = x2[DATE].dt.dayofweek
    x2["is_nonhol_weekday"] = ((x2["dow"]<5) & (~x2[DATE].isin(hol_set))).astype(int)
    x2["is_hol_wk"] = x2[DATE].isin(hol_set).astype(int)

    outs = []
    for k, g in x2.groupby(KEY, sort=False):
        if (BRUNCH_WD in str(k)) or (BRUNCH_WE in str(k)):
            outs.append(g); continue
        g = g.copy()
        idx = (g["is_nonhol_weekday"]==1)
        s = g.loc[idx, TARGET].astype(float)
        med = s.shift(1).rolling(window=4, min_periods=2).median()
        g.loc[idx, "med_wd_dow"] = med.values
        is_metric_target = (("단체" in str(k)) or (str(k) in EXTRA_GROUP_KEYS))
        if is_metric_target:
            thr_up = 1.25 * g["med_wd_dow"]
            adj = (g["is_hol_wk"]==1) & np.isfinite(g["med_wd_dow"]) & (g[TARGET] > thr_up)
            g.loc[adj, TARGET] = np.maximum(0.0, np.round(1.10 * g.loc[adj, "med_wd_dow"]))
        outs.append(g.drop(columns=["med_wd_dow"]))
    return pd.concat(outs, axis=0).drop(columns=["dow","is_nonhol_weekday","is_hol_wk"])

In [22]:
def _read_csv_robust(p):
    try:    return pd.read_csv(p)
    except UnicodeDecodeError:
        return pd.read_csv(p, encoding="cp949")

test_wday = {}
for wp in sorted(glob.glob("test/meta/TEST_weather_*.csv")):
    wid = os.path.splitext(os.path.basename(wp))[0].split("_")[-1]  # "00"~"09"
    fid = f"TEST_{wid}"  # TEST_00 …

    w = _read_csv_robust(wp).rename(columns={"일시": DATE})
    w[DATE] = to_datetime_norm(w[DATE])
    col_tmax = next(c for c in w.columns if ("최고" in c and "기온" in c))
    col_tavg = next(c for c in w.columns if ("평균" in c and "기온" in c))
    w_day_test = (w[[DATE, col_tmax, col_tavg]]
                  .groupby(DATE, as_index=False).mean(numeric_only=True)
                  .rename(columns={col_tmax:"wx_tmax", col_tavg:"wx_tavg"}))
    test_wday[fid] = w_day_test

_w_fallback = pd.DataFrame({DATE: pd.to_datetime([]), "wx_tmax": [], "wx_tavg": []})

def add_weather_feats(df, wday):
    return df.merge(wday, on=DATE, how="left")  # wday는 (DATE, wx_tmax, wx_tavg)

test_room = {}
for wp in sorted(glob.glob("test/meta/TEST_room_*.csv")):# wp : test/meta/TEST_room_09.csv
    wid = os.path.splitext(os.path.basename(wp))[0].split("_")[-1]  # "00"~"09"
    fid = f"TEST_{wid}"  # TEST_00 …

    w = _read_csv_robust(wp).rename(columns={"일시": DATE}) # room_m data
    w[DATE] = to_datetime_norm(w[DATE])

    today_num = []

    for i in range(len(w)):
        a = w.iloc[i]
        _sum = 0
        for _type in room_dict.keys():
            _sum += a[_type]*room_dict[_type]

        today_num.append(_sum)

    w['총방문객수'] = today_num

    test_room.update({fid:w})

_room_fallback = pd.DataFrame({DATE: pd.to_datetime([]), "총방문객수": []})

def add_room_feats(df, room):
    return df.merge(room, on=DATE, how="left")  # room은 (DATE, 총방문객수)

def to_datetime_norm(s):
    return pd.to_datetime(s, errors="coerce").dt.normalize()

test_group = {}

for gp in sorted(glob.glob("test/meta/TEST_group_*.csv")):  # ex: test/meta/TEST_group_09.csv
    wid = os.path.splitext(os.path.basename(gp))[0].split("_")[-1]  # "00" ~ "09"
    fid = f"TEST_{wid}"  # TEST_00 …

    g = _read_csv_robust(gp).rename(columns={"영업일자": DATE})
    g[DATE] = to_datetime_norm(g[DATE])

    # 겨울 시즌 피처 생성
    g["month"] = g[DATE].dt.month
    g["is_ski_season"] = g["month"].isin([12,1,2]).astype(int)

    season_mask = g["is_ski_season"] == 1
    if season_mask.any():
        season_start = g.loc[season_mask, DATE].min()
        season_end   = g.loc[season_mask, DATE].max()
        season_len   = (season_end - season_start).days + 1
        g["day_of_season"] = (
            (g[DATE] - season_start).dt.days / season_len
        ).where(season_mask, 0.0)
    else:
        g["day_of_season"] = 0.0

    # 필요한 컬럼만 저장
    test_group[fid] = g[[DATE, "is_ski_season", "day_of_season"]]

# fallback (merge 실패 시 빈 df 반환용)
_group_fallback = pd.DataFrame({DATE: pd.to_datetime([]),
                                "is_ski_season": [],
                                "day_of_season": []})

def add_group_feats(df, gday):
    return df.merge(gday, on=DATE, how="left")

test_group1 = {}

for gp in sorted(glob.glob("test/meta/TEST_group_*.csv")):  
    wid = os.path.splitext(os.path.basename(gp))[0].split("_")[-1]  
    fid = f"TEST_{wid}"

    g = _read_csv_robust(gp).rename(columns={"영업일자": DATE})
    g[DATE] = pd.to_datetime(g[DATE]).dt.normalize()

    # 카페테리아 + 포레스트릿 합산
    col_a = next(c for c in g.columns if "카페테리아" in c)
    col_b = next(c for c in g.columns if "포레스트릿" in c)
    g["cf_sum"] = g[col_a] + g[col_b]

    # 이진화
    g["is_forest_event"] = (g["cf_sum"] > 0).astype(int)

    # dict에 저장
    test_group1[fid] = g[[DATE, "is_forest_event"]]

_group_fallback_2 = pd.DataFrame({
    DATE: pd.to_datetime([]),
    "is_forest_event": []
})

def add_group_feats(df, gday):
    return df.merge(gday, on=DATE, how="left")
    
# ==== 추가 끝 ====

def build_feats(df_in):
    f = df_in.copy()
    f = f.sort_values([KEY, DATE])
    f["month"] = f[DATE].dt.month
    f["dayofweek"] = f[DATE].dt.dayofweek
    f["day"] = f[DATE].dt.day
    f["dayofyear"] = f[DATE].dt.dayofyear
    f["is_month_end"] = f[DATE].dt.is_month_end.astype(int)
    f["is_quarter_end"] = f[DATE].dt.is_quarter_end.astype(int)
    f["is_weekend"] = (f["dayofweek"]>=5).astype(int)

    for h in range(1, PREDICT+1):
        dth = f[DATE] + pd.to_timedelta(h, unit="D")
        f[f"dow_h{h}"] = dth.dt.dayofweek
        f[f"is_weekend_h{h}"] = (f[f"dow_h{h}"]>=5).astype(int)
        f[f"is_wkhol_h{h}"] = dth.isin(pd.DatetimeIndex(HOL_WEEKDAY_SET)).astype(int)

    for lag in [7,14,21]: f[f"lag_{lag}"] = f.groupby(KEY)[TARGET].shift(lag)
    for lag in [1,2,3,4,5,6]: f[f"lag_{lag}"] = f.groupby(KEY)[TARGET].shift(lag)
    for w in [7,14]:
        f[f"rolling_mean_{w}"]   = f.groupby(KEY)[TARGET].transform(lambda x: x.shift(1).rolling(w).mean())
        f[f"rolling_std_{w}"]    = f.groupby(KEY)[TARGET].transform(lambda x: x.shift(1).rolling(w).std())
        f[f"rolling_median_{w}"] = f.groupby(KEY)[TARGET].transform(lambda x: x.shift(1).rolling(w).median())
    f["rolling_mean_21"] = f.groupby(KEY)[TARGET].transform(lambda x: x.shift(1).rolling(21).mean())
    f["ratio_mean7_21"]  = f["rolling_mean_7"]/f["rolling_mean_21"]
    f["ratio_mean14_21"] = f["rolling_mean_14"]/f["rolling_mean_21"]

    def _slope(v):
        v = v[np.isfinite(v)]
        if v.size < 2: return np.nan
        x = np.arange(v.size, dtype=float)
        xm, ym = x.mean(), v.mean()
        denom = ((x-xm)**2).sum()
        return 0.0 if denom==0 else float(((x-xm)*(v-ym)).sum()/denom)

    f["slope_7"]  = f.groupby(KEY)[TARGET].transform(lambda x: x.shift(1).rolling(7).apply(_slope, raw=True))
    f["slope_14"] = f.groupby(KEY)[TARGET].transform(lambda x: x.shift(1).rolling(14).apply(_slope, raw=True))
    f["ewm_mean_7"] = f.groupby(KEY)[TARGET].transform(lambda x: x.shift(1).ewm(span=7, adjust=False, min_periods=2).mean())
    x_dow = f["dayofweek"].astype(float)
    f["dow_sin"] = np.sin(2*np.pi*x_dow/7.0); f["dow_cos"] = np.cos(2*np.pi*x_dow/7.0)
    f["key_encoded"] = f[KEY].astype(str).map(key2id).astype("category")

    def _flag_lzb_local(g):
        iz = (g[TARGET]==0).astype(int)
        bid = (iz != iz.shift()).cumsum()
        blen = iz.groupby(bid).transform("sum")
        return ((iz==1) & (blen >= NON_SELLING_MIN_DAYS)).astype(int)
    f["is_long_zero_block"] = f.groupby(KEY, sort=False).apply(_flag_lzb_local).reset_index(level=0, drop=True).astype(int)
    return f

# === 앵커 앙상블 + p 앙상블 동시 적용 ===
all_preds = []
for path in test_files:
    fid = os.path.splitext(os.path.basename(path))[0]
    tdf = _read_csv_robust(path)
    tdf[DATE] = to_datetime_norm(tdf[DATE])
    tdf = tdf.sort_values([KEY, DATE]).reset_index(drop=True)

    tdf_imp  = weekday_holiday_impute(tdf, HOL_WEEKDAY_SET)
    tdf_safe = spike_clamp(tdf_imp, HOL_WEEKDAY_SET)
    tdf_safe = add_weather_feats(tdf_safe, test_wday.get(fid, _w_fallback))
    tdf_safe = add_room_feats(tdf_safe, test_room.get(fid, _room_fallback))
    tdf_safe = add_group_feats(tdf_safe, test_group.get(fid, _group_fallback))
    tdf_safe = add_group_feats(tdf_safe, test_group1.get(fid, _group_fallback_2))
    
    his = tdf_safe.sort_values([KEY, DATE])
    last_date = his[DATE].max()
    date_set = set(his[DATE])
    anchors = sorted(d for d in (last_date - pd.to_timedelta(np.arange(ANCHOR_K_FULL), "D")) if d in date_set)

    bucket = {}
    for ad in anchors:
        sub = his[his[DATE] <= ad].copy()
        Feat = build_feats(sub)
        rowA = Feat[Feat[DATE]==ad].copy()
        if rowA.empty: 
            continue
        Xrow = rowA[feat_cols].copy()
        for c in ("key_encoded","month","dayofweek"):
            if c in Xrow.columns: Xrow[c] = Xrow[c].astype("category")

        # p-가중 앙상블 예측
        Y_mix = None
        for p, mdl in models_full.items():
            Yp = mdl.predict(Xrow)  # (n_keys, 7)
            w  = P_WEIGHTS.get(p, 0.0)
            Y_mix = (w*Yp) if Y_mix is None else (Y_mix + w*Yp)

        meta = rowA[[KEY,DATE]].reset_index(drop=True)
        for i in range(len(meta)):
            keyi = meta.at[i, KEY]
            for h in range(PREDICT):
                dtp = (ad + pd.Timedelta(days=h+1)).normalize()
                bucket.setdefault((dtp, keyi), []).append(float(Y_mix[i, h]))

    for (dtp, keyi), vals in bucket.items():
        all_preds.append({DATE: dtp, KEY: keyi, TARGET: float(np.mean(vals))})

In [23]:
Xrow

,month,dayofweek,key_encoded,day,dayofyear,is_month_end,is_quarter_end,is_weekend,rolling_mean_7,rolling_std_7,...,is_wkhol_h4,dow_h5,is_weekend_h5,is_wkhol_h5,dow_h6,is_weekend_h6,is_wkhol_h6,dow_h7,is_weekend_h7,is_wkhol_h7
27,5,5,0,24,144,0,0,1,3.571429,5.094348,...,0,3,0,0,4,0,0,5,1,0
55,5,5,1,24,144,0,0,1,40.142857,45.772730,...,0,3,0,0,4,0,0,5,1,0
83,5,5,2,24,144,0,0,1,5.428571,4.961759,...,0,3,0,0,4,0,0,5,1,0
111,5,5,3,24,144,0,0,1,2.428571,4.314979,...,0,3,0,0,4,0,0,5,1,0
139,5,5,4,24,144,0,0,1,0.428571,1.133893,...,0,3,0,0,4,0,0,5,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4563,5,5,162,24,144,0,0,1,29.142857,20.497387,...,0,3,0,0,4,0,0,5,1,0
4591,5,5,163,24,144,0,0,1,31.000000,24.812631,...,0,3,0,0,4,0,0,5,1,0
4619,5,5,164,24,144,0,0,1,85.714286,48.016366,...,0,3,0,0,4,0,0,5,1,0
4647,5,5,165,24,144,0,0,1,22.142857,13.716518,...,0,3,0,0,4,0,0,5,1,0


## 7. 후처리(Metric-aware A+B, 라운딩, min-1 규칙)

In [24]:
# === [대체 셀] 기존 후처리 로직 중 '블록 통계 소스' 불일치만 수정 (tdf_safe 기준으로 재계산) ===

sub_long = pd.DataFrame(all_preds).copy()
sub_long[DATE] = to_datetime_norm(sub_long[DATE])

# 파일별 블록 통계 수집: 예측 단계와 동일한 안전 전처리(tdf_safe) 기준으로 pos/length/weekend_mean 산출
date_to_file, block_pos_values, block_lengths, weekend_means = {}, {}, {}, {}
for p in test_files:
    fid = os.path.basename(p).split(".")[0]
    try:
        dfb = pd.read_csv(p)
    except UnicodeDecodeError:
        dfb = pd.read_csv(p, encoding="cp949")
    dfb[DATE] = to_datetime_norm(dfb[DATE])
    dfb = dfb.sort_values([KEY, DATE]).reset_index(drop=True)

    # (A) 평일 공휴일 imputing  [예측 단계와 동일]
    xx = dfb.copy()
    xx["dow"] = xx[DATE].dt.dayofweek
    xx["is_weekday"] = (xx["dow"] < 5).astype(int)
    xx["is_weekday_hol"] = xx[DATE].isin(HOL_WEEKDAY_SET).astype(int)
    outs = []
    for key_i, g in xx.groupby(KEY, sort=False):
        g = g.sort_values(DATE).copy()
        end_date = g[DATE].max()
        cand_all = g[(g["is_weekday"] == 1) & (g["is_weekday_hol"] == 0)].copy()
        all_weekday_zero = (cand_all[TARGET].sum() == 0)
        cand = cand_all
        for idx, row in g.iterrows():
            if not (row["is_weekday"] == 1 and row["is_weekday_hol"] == 1):
                continue
            if all_weekday_zero:
                g.at[idx, TARGET] = 0
                continue
            dow_h = int(row["dow"]); date_h = row[DATE]
            pool_dow = cand[cand["dow"] == dow_h]
            def _wmean(vals, dates, end_date, decay=0.90, lastweek_window=7, boost=0.50):
                if len(vals) == 0: return np.nan
                vals = np.asarray(vals, float); dates = pd.to_datetime(dates)
                delta = (end_date - dates).days.astype(float)
                w = (decay ** delta)
                w = np.where(dates >= (end_date - pd.Timedelta(days=lastweek_window-1)), w*(1.0+boost), w)
                w = np.clip(w, 1e-6, None)
                return float(np.sum(w*vals) / np.sum(w))
            base = _wmean(pool_dow[TARGET].values, pool_dow[DATE].values, end_date=end_date) if len(pool_dow) >= 1 \
                   else _wmean(cand[TARGET].values, cand[DATE].values, end_date=end_date)
            t7 = date_h - pd.Timedelta(days=7)
            prev = g[(g[DATE] == t7) & (g["is_weekday"] == 1) & (g["is_weekday_hol"] == 0)]
            val_t7 = float(prev[TARGET].iloc[0]) if len(prev) == 1 else np.nan
            if np.isfinite(base):
                y_hat = base if not np.isfinite(val_t7) else 0.65*base + 0.35*val_t7
                g.at[idx, TARGET] = int(max(0, round(y_hat)))
            else:
                g.at[idx, TARGET] = 0
        outs.append(g)
    tdf_imp_stat = pd.concat(outs, axis=0).drop(columns=["dow","is_weekday","is_weekday_hol"])

    # (B) 급등 클램프  [예측 단계와 동일]
    x2 = tdf_imp_stat.copy()
    x2["dow"] = x2[DATE].dt.dayofweek
    x2["is_nonhol_weekday"] = ((x2["dow"] < 5) & (~x2[DATE].isin(HOL_WEEKDAY_SET))).astype(int)
    x2["is_hol_wk"] = (x2[DATE].isin(HOL_WEEKDAY_SET)).astype(int)
    outs = []
    for k_i, g in x2.groupby(KEY, sort=False):
        if (BRUNCH_WD in str(k_i)) or (BRUNCH_WE in str(k_i)):
            outs.append(g); continue
        g = g.copy()
        g["med_wd_dow"] = np.nan
        for d in range(5):
            idx = (g["dow"] == d) & (g["is_nonhol_weekday"] == 1)
            s = g.loc[idx, TARGET].astype(float)
            med = s.shift(1).rolling(window=4, min_periods=2).median()
            g.loc[idx, "med_wd_dow"] = med.values
        is_metric_target = (("단체" in str(k_i)) or (str(k_i) in EXTRA_GROUP_KEYS))
        if is_metric_target:
            thr_up = (1.0 + 0.25) * g["med_wd_dow"]
            adj = ((g["is_hol_wk"] == 1) & np.isfinite(g["med_wd_dow"]) & (g[TARGET] > thr_up))
            g.loc[adj, TARGET] = np.maximum(0.0, np.round((1.0 + 0.10) * g.loc[adj, "med_wd_dow"]))
        g = g.drop(columns=["med_wd_dow"])
        outs.append(g)
    tdf_safe_stat = pd.concat(outs, axis=0).drop(columns=["dow","is_nonhol_weekday","is_hol_wk"]).sort_values([KEY, DATE])

    # tdf_safe 기준 통계
    last = tdf_safe_stat[DATE].max()
    for h in range(1, PREDICT+1):
        date_to_file[(last + pd.Timedelta(days=h)).strftime("%Y-%m-%d")] = fid
    for k_i, g in tdf_safe_stat.groupby(KEY):
        g = g.sort_values(DATE)
        pos = g.loc[g[TARGET] > 0, TARGET].astype(float).values
        block_pos_values[(fid, k_i)] = pos
        block_lengths[(fid, k_i)] = int(g.shape[0])
        wknd = g[g[DATE].dt.dayofweek >= 5][TARGET].astype(float)
        weekend_means[(fid, k_i)] = float(wknd.mean()) if wknd.size > 0 else np.nan

# (A) 평일 공휴일이면 주말 평균으로 대체  [변경 없음]
if APPLY_METRIC_AWARE_WK_HOLOV:
    rows = []
    for _, r in sub_long.iterrows():
        dt = r[DATE]
        if (dt.weekday() < 5) and (dt in HOL_WEEKDAY_SET):
            fid = date_to_file.get(dt.strftime("%Y-%m-%d"))
            wk = weekend_means.get((fid, r[KEY])) if fid is not None else np.nan
            val = float(wk) if np.isfinite(wk) else float(r[TARGET])
        else:
            val = float(r[TARGET])
        rows.append(val)
    sub_long[TARGET] = np.array(rows, dtype=float)

# (B) weighted-SMAPE 블렌딩  [변경 없음]
def is_metric_target_post(k):
    s = str(k)
    if "공깃밥" in s: return False
    if (BRUNCH_WD in s) or (BRUNCH_WE in s): return False
    if "단체" in s: return True
    return s in EXTRA_GROUP_KEYS

def _weighted_smape_constant(pos_vals, n_days, decay=0.92, clamp_k=0.50, beta_soft=0.50):
    pos = np.asarray(pos_vals, float)
    if pos.size == 0: return 1, 0.0
    pos = np.sort(pos); q1,q2,q3 = np.percentile(pos,[25,50,75]); iqr = q3-q1; p95 = np.percentile(pos,95)
    w = np.array([decay**(pos.size-1-i) for i in range(pos.size)], float); w /= w.sum()
    base_cands = np.unique(np.round(np.r_[pos,q1,q2,q3,(q1+q3)/2,p95]).astype(int))
    lo = max(1, int(np.floor(max(1, q1 - clamp_k*iqr)))); hi = int(np.ceil(min(max(base_cands.max(),2), p95*1.10)))
    cands = np.unique(np.r_[base_cands, np.arange(lo, max(lo+1, hi+1))])
    def _wsmape(c): c=float(c); return np.sum(w*(2.0*np.abs(c-pos)/(np.abs(c)+pos)))
    scores = np.array([_wsmape(c) for c in cands]); c_star = int(cands[np.argmin(scores)])
    clamp_lo = max(1, int(np.floor(q1 - clamp_k*iqr))); clamp_hi = int(np.ceil(q3 + clamp_k*iqr))
    c_clip = int(np.clip(c_star, clamp_lo, clamp_hi))
    c_final = int(np.round((1 - beta_soft)*c_star + beta_soft*c_clip)); c_final = max(1, c_final)
    zero_rate = 1.0 - (pos.size / max(1, n_days))
    iqr_ratio = (iqr / (q2 + 1e-9)) if q2 > 0 else 0.0; iqr_ratio = float(np.clip(iqr_ratio, 0.0, 2.0))
    alpha_raw = 0.6*zero_rate + 0.4*(iqr_ratio / (1.0 + iqr_ratio))
    alpha = float(np.clip(alpha_raw, 0.0, 0.5))
    return c_final, alpha

if APPLY_METRIC_AWARE_WSMAPE:
    new_vals = []
    for _, r in sub_long.iterrows():
        k = r[KEY]
        if not is_metric_target_post(k):
            new_vals.append(float(r[TARGET])); continue
        fid = date_to_file.get(r[DATE].strftime("%Y-%m-%d"))
        if fid is None:
            new_vals.append(float(r[TARGET])); continue
        pos = block_pos_values.get((fid, k), np.array([]))
        nd  = block_lengths.get((fid, k), INPUT_WINDOW_DAYS)
        c, a = _weighted_smape_constant(pos, nd)
        y = float(r[TARGET]); y_new = (1.0 - a)*y + a*float(c)
        new_vals.append(y_new)
    sub_long[TARGET] = np.array(new_vals, dtype=float)

# 라운딩 임계값은 그대로 유지
ths = np.round(np.arange(0.25, 0.351, 0.01), 3)
best_thr = 0.130
sub_long[TARGET] = round_nonneg(sub_long[TARGET].values, best_thr).astype(int)

# 활성 키 min-1  [변경 없음]
if FORCE_ACTIVE_MIN1 or FORCE_GLOBAL_MIN1:
    if FORCE_ACTIVE_MIN1:
        upd = []
        for _, r in sub_long.iterrows():
            y = int(r[TARGET])
            fid = date_to_file.get(r[DATE].strftime("%Y-%m-%d"))
            if (y == 0) and (fid is not None) and is_metric_target_post(r[KEY]):
                pos = block_pos_values.get((fid, r[KEY]), np.array([]))
                if pos.size > 0: y = 1
            upd.append(y)
        sub_long[TARGET] = np.array(upd, dtype=int)
    if FORCE_GLOBAL_MIN1:
        sub_long.loc[sub_long[TARGET] == 0, TARGET] = 1

print("postprocessed rows:", len(sub_long))
sub_long.head(3)

postprocessed rows: 21710


,영업일자,영업장명_메뉴명,매출수량
0,2024-07-08,느티나무 셀프BBQ_1인 수저세트,1
1,2024-07-09,느티나무 셀프BBQ_1인 수저세트,2
2,2024-07-10,느티나무 셀프BBQ_1인 수저세트,3


## 8. 제출 파일 생성

In [25]:
OUT_PATH = "Pipeline_1_we_PEsemble_cs3.csv"

# sample 형식에 맞춰 pivot
sample = pd.read_csv("sample_submission.csv", engine="python")
label_col = sample.columns[0]

# date_label 매핑: "TEST_xx+Nd"
date_label = {}
for fp in test_files:
    tdf = pd.read_csv(fp)
    tdf[DATE] = to_datetime_norm(tdf[DATE])
    fid = os.path.basename(fp).split(".")[0]
    last = tdf[DATE].max()
    for i in range(PREDICT):
        date_label[(last + pd.Timedelta(days=i+1)).strftime("%Y-%m-%d")] = f"{fid}+{i+1}일"

sub_long = sub_long.copy()
sub_long["date_label"] = sub_long[DATE].dt.strftime("%Y-%m-%d").map(date_label)

pivot = sub_long.pivot_table(index="date_label", columns=KEY, values=TARGET, aggfunc="first")
final = sample.set_index(label_col)
common = [c for c in final.columns if c in pivot.columns]
if len(common)>0:
    final[common] = final[common].combine_first(pivot[common])
    final.update(pivot[common])

final = final.fillna(1).astype(np.int64).reset_index()
final.columns = sample.columns

final.to_csv(OUT_PATH, index=False, encoding="utf-8-sig")
print("[SAVE] ->", OUT_PATH)
final.head(5)

[SAVE] -> Pipeline_1_we_PEsemble_cs3.csv


,영업일자,느티나무 셀프BBQ_1인 수저세트,느티나무 셀프BBQ_BBQ55(단체),"느티나무 셀프BBQ_대여료 30,000원","느티나무 셀프BBQ_대여료 60,000원","느티나무 셀프BBQ_대여료 90,000원",느티나무 셀프BBQ_스프라이트 (단체),느티나무 셀프BBQ_신라면,느티나무 셀프BBQ_쌈장,느티나무 셀프BBQ_육개장 사발면,...,화담숲주막_스프라이트,화담숲주막_참살이 막걸리,화담숲주막_찹쌀식혜,화담숲주막_콜라,화담숲주막_해물파전,화담숲카페_메밀미숫가루,화담숲카페_아메리카노 HOT,화담숲카페_아메리카노 ICE,화담숲카페_카페라떼 ICE,화담숲카페_현미뻥스크림
0,TEST_00+1일,6,56,6,3,1,14,2,1,1,...,5,12,12,4,46,20,5,22,6,10
1,TEST_00+2일,1,58,1,1,1,15,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2,TEST_00+3일,3,60,2,1,1,15,1,1,1,...,3,5,5,2,17,10,3,11,4,3
3,TEST_00+4일,4,60,2,1,1,16,1,1,1,...,2,4,5,2,16,9,3,11,3,3
4,TEST_00+5일,6,63,3,2,1,18,2,1,1,...,2,5,6,2,16,12,3,15,5,4


In [26]:
final.shape

(70, 168)